In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import glob
import re

In [6]:
data_dir = "../private_data/"
data_fps = glob.glob(os.path.join(data_dir, "*.csv"))

In [ ]:
df = pd.read_csv(data_fps[0])
df = pd.concat([df, pd.DataFrame({"has_corrected_error": [4], "has_bugcheck": [4]})], ignore_index=True)
df.tail()

In [48]:
def nonprivate_analysis(data_fp):
    df = pd.read_csv(data_fp)
    
    if df.groupby(["has_corrected_error", "has_bugcheck"]).size().shape[0] != 4:
        df = pd.concat([df, pd.DataFrame({"has_corrected_error": [1], "has_bugcheck": [0]})], ignore_index=True)
        df = pd.concat([df, pd.DataFrame({"has_corrected_error": [1], "has_bugcheck": [1]})], ignore_index=True)
        
    X = df[["has_corrected_error"]]
    X = sm.add_constant(X)
    y = df["has_bugcheck"]
    
    model = sm.Logit(y, X)
    result = model.fit()
    return {
        "pvalue": result.pvalues.iloc[1],
        "coef": result.params.iloc[1],
        "llf": result.llf
    }  


In [ ]:
results = {
    int(re.findall(r'bugcheck_(\d+)', data_fp)[0]) : nonprivate_analysis(data_fp)
    for data_fp in data_fps
}

In [ ]:
pd.DataFrame(results).T